In [1]:
import os
import PyPDF2
import markdown2
import json
import pickle
import pandas as pd

from docx import Document as DocxDocument
from pptx import Presentation

from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings.base import OpenAIEmbeddings

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Rinats well documented sample project
# https://github.com/trustbit/private-poc-fmw-content-generator/tree/main/backend-and-kb/src/fmw

from dotenv import load_dotenv

load_dotenv()

True

## Get Companies

In [2]:
SAMPLES_PATH = r"C:\Users\felix.krause\code\trustbit\enterprise-rag-challenge\samples"

In [3]:
df = pd.read_csv("data/dataset.csv") 
df

,sha1,date,name,size
0,ce9e5024041b2ece2bafa2a9d9516bb174ee8949,2022-10-31,"Anixa Biosciences, Inc.",3996701
1,f71415f9ca0cff70e5fa193616b6197f361130ed,2023-02-21,"Maravai LifeSciences Holdings, Inc.",4033642
2,4a9d2b853e05970776121a810460f0962a18c5a1,2022-XX,KLA Corporation,1181894
3,f973dd219c534accb0d4e72d8e12f51284d48d10,2023-01-01,"Ameresco, Inc.",10648267
4,4e27f4c3402c657d548760cb3a164b036cefaabb,2022-12-31,Battery Minerals Limited,3650701
...,...,...,...,...
7490,a2afcd8165a6dbd0058682680b65d3638a5800eb,2023-02-02,"Arrow Electronics, Inc",922750
7491,215df84494756bd4feebc973657835ef7f14ee16,2022-12-31,Synertec,10474497
7492,20fb970d8705289e835b408c575351295ac16f5f,2022-09-30,TE Connectivity,5753097
7493,c7f3a8c0a38c756438950ce3085076adb4241a32,2022-01-01,OTC_ADDDF,12603054


In [4]:
# load all file names in the folder
sha1_dict_full = df.set_index("sha1").to_dict()["name"]

included_companies = []

for file in os.listdir(SAMPLES_PATH):
    file = file[:-4]
    try:
        included_companies.append(sha1_dict_full[file])
    except KeyError:
        print(file)

questions.


In [122]:
included_companies

['Global Medical REIT Inc.',
 'Zegona Communications plc',
 'Calyxt, Inc.',
 'TD SYNNEX',
 'Eurocell PLC',
 'Sandwell Aquatics Centre',
 'Mercurity Fintech Holding Inc.',
 'Motus GI Holdings, Inc.',
 'Limbach Holdings, Inc.',
 'Strike Energy Limited',
 'Platform Technology',
 'VENUS METALS CORPORATION LIMITED',
 'NICE',
 'Nykredit',
 'Air Products',
 'Accuray Incorporated',
 'Kooth Plc',
 'Ethernity Networks Ltd',
 'Canadian Banc Corp.',
 'EMT']

In [6]:
# Create sample for development
# COMPANIES = ["Ethernity Networks Ltd", "Limbach Holdings, Inc.", "Accuray Incorporated"]
COMPANIES = included_companies

if COMPANIES:
    df_sample = df[df.name.isin(COMPANIES)]
else:
    df_sample = df.copy()
    
sha1_dict = df_sample.set_index("name").to_dict()["sha1"]

In [7]:
df_sample

,sha1,date,name,size
267,d81bbc64a4160b9946fea7a895f80e6201f52f27,2022-11-23,Air Products,4889969
981,608c5097dfc6e83505fd2259ad862dcec11a3f96,2022-12-31,Sandwell Aquatics Centre,2661906
1075,3696c1b29566acc1eafc704ee5737fb3ae6f3d1d,2023-01-01,Zegona Communications plc,1481844
1183,99be213e4e689294ebae809bfa6a1b5024076286,2022-01-01,"Limbach Holdings, Inc.",2023552
1324,71b04e0248ecf758990a0ab77bd69344be63bcf4,2023-03-27,"Motus GI Holdings, Inc.",7725729
1500,6b79f1c1de9d0e39a4576dcd4585849b9465b402,2023-04-05,Mercurity Fintech Holding Inc.,1767320
2633,40b5cfe0d7bbf59e186492bfbe1b5002d44af332,2023-03-01,"Calyxt, Inc.",694201
2838,faf8d7d79152d61279eda1cfb58b8236ce2f82fa,2022-06-30,EMT,1360293
3270,4b525836a5d7cb75489f6d93a3b1cf2b8f039bf2,2022-10-01,TD SYNNEX,1902884
4263,ea0757d27fa67cd347d9f046b939a911f5c9a08d,2022-01-01,Canadian Banc Corp.,498408


## Get company name from user query

TODO could be more sophisticated

In [10]:
import re
from fuzzywuzzy import fuzz

def normalize_name(name):
    # Normalize by removing extra spaces and converting to lowercase
    return re.sub(r'\s+', ' ', name.strip().lower())

def fuzzy_match(companies, input_string, threshold=85):
    matched_companies = []
    norm_input = normalize_name(input_string)
    
    for company in companies:
        norm_company = normalize_name(company)
        prev_token = ""
        
        # Fuzzy match each token
        for curr_token in norm_input.split():
            token = prev_token + " " + curr_token
            
            match_score = fuzz.ratio(norm_company, token)
            prev_token = curr_token
            
            if match_score >= threshold:
                matched_companies.append(company)
                break  # Stop once a match is found for this company
            
    return matched_companies


def find_exact_matches(companies, input_string):
    # normalized_input = normalize_name(input_string)
    
    matched_companies = []
    
    for company in companies:
        if company in input_string:
            matched_companies.append(company)
    
    return matched_companies


def find_companies(input_string):
    # Wrapper to use later and allow easy function change
    return find_exact_matches(df.name, input_string)


input_string = ("limbach holding inc and ethernity networks buys the accuracy incorporated, ESEA, Five9, Inc.")

matches = find_exact_matches(df.name, input_string)
print(matches)

['EA', 'ESEA', 'Five9, Inc.']


C:\Users\felix.krause\code\trustbit\llm_experiments\venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Load LLM data

In [162]:
from llama_parse import LlamaParse

parser = LlamaParse(
    result_type="markdown",  # "markdown" and "text" are available
    verbose=True
)

# Function to load text from different file types
def load_text_from_file(file_path):
    _, file_extension = os.path.splitext(file_path)
    text = ""
    
    if file_extension == ".pdf":
        # TODO https://www.reddit.com/r/LangChain/comments/18yxacm/extracting_data_from_pdf_containing_complex_tables/
        with open(file_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f, strict=False)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    else:
        pass
    
    return text


# Function to load documents from a folder
def load_documents_from_folder(folder_path, companies=[]):
    documents = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file[:-4] in companies: #or len(companies) == 0:
                print("Loading", file)
                file_path = os.path.join(root, file)

                text = load_text_from_file(file_path)
                if text:
                    documents.append(Document(page_content=text, metadata={"source": file_path}))
    return documents

In [12]:
# Could try LLama approach
# TODO https://medium.com/the-ai-forum/rag-on-complex-pdf-using-llamaparse-langchain-and-groq-5b132bd1f9f3
# doc = parser.load_data(os.path.join(FOLDER_PATH, "e51b7204b91cbe7709bd3218e7d2d0c2b8dbb438.pdf"))
# index = VectorStoreIndex.from_documents(doc)

## Try to detect tables from pdf

In [8]:
"""
import camelot

# Extract tables from a PDF file
tables = camelot.read_pdf(os.path.join(FOLDER_PATH, "e51b7204b91cbe7709bd3218e7d2d0c2b8dbb438.pdf"), pages="all")

# Convert the first table to a DataFrame
df = tables[0].df

# Save the DataFrame as a CSV file
tables[0]
"""

C:\Users\felix.krause\code\trustbit\llm_experiments\venv\lib\site-packages\camelot\parsers\lattice.py:397: UserWarning: No tables found on page-2
  warnings.warn(
C:\Users\felix.krause\code\trustbit\llm_experiments\venv\lib\site-packages\camelot\parsers\lattice.py:397: UserWarning: No tables found on page-79
  warnings.warn(


<Table shape=(1, 1)>

In [47]:
# tables[40].df

,0,1,2,3
0,STRATEGIC REPORT\nCORPORATE GOVERNANCE,,FINANCIAL STATEMENTS\nFINANCIAL STATEMENTS,73\n73


## Build retriever per company

In [163]:
def get_store_name(string):
    string = string.lower()
    return string.replace(" ", "_").replace(",", "").replace(".", "").replace("'", "")

In [12]:
# WARNING: This will build all retrievers for all companies in the sample
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=300, separators=["\n\n", "\n"]) # TODO more sophisticated

for company in df_sample.name.to_list():
    # check if file already exists
    if os.path.exists(f"dbs/{get_store_name(company)}.db"):
        print("Skipping", company)
        continue
    
    try:
        company_docs = load_documents_from_folder(SAMPLES_PATH, [sha1_dict[company]])
        
        if not company_docs:
            raise ValueError(f"No documents found for company: {company}")
        
        texts = text_splitter.split_documents(company_docs)
        embeddings = OpenAIEmbeddings()
        db = FAISS.from_documents(texts, embeddings)
        
        with open(f"dbs/{get_store_name(company)}.db", "wb") as f:
            pickle.dump(db.serialize_to_bytes(), f)
    except Exception as e:
        print(f"Error for company {company}: {e}")
        # raise e

Loading d81bbc64a4160b9946fea7a895f80e6201f52f27.pdf
Loading 608c5097dfc6e83505fd2259ad862dcec11a3f96.pdf
Loading 3696c1b29566acc1eafc704ee5737fb3ae6f3d1d.pdf


FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead


Loading 99be213e4e689294ebae809bfa6a1b5024076286.pdf


FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead
FloatObject (b'0.00-40') invalid; use 0.0 instead


Loading 71b04e0248ecf758990a0ab77bd69344be63bcf4.pdf
Loading 6b79f1c1de9d0e39a4576dcd4585849b9465b402.pdf
Loading 40b5cfe0d7bbf59e186492bfbe1b5002d44af332.pdf
Loading faf8d7d79152d61279eda1cfb58b8236ce2f82fa.pdf
Loading 4b525836a5d7cb75489f6d93a3b1cf2b8f039bf2.pdf
Loading ea0757d27fa67cd347d9f046b939a911f5c9a08d.pdf
Loading 9e703e719d94af786af5511c823ff86e9f04c070.pdf
Loading dfb1e552b18e116105d9125d9becafa443950e97.pdf
Loading e51b7204b91cbe7709bd3218e7d2d0c2b8dbb438.pdf
Loading bd5041c3e6909d92a7a88e4fb10dd8651df33228.pdf
Loading 58a5f9f5c83159e63602b0b1dd27c27fb945c0e9.pdf
Loading 9ff4e041732c9841d5423e6ea0bbd6a0320df9ff.pdf
Loading dd78f748262b8ffa62de6484143ff55b38af24c7.pdf
Loading d734bac4a4815e616d84083ad4d3844655321215.pdf
Loading 9ae3bb21564a5098c4b4d6450655c22eff85deae.pdf
Loading 053b7cb83115789346e2a9efc7e2e640851653ff.pdf


In [164]:
def get_retriever_dict(companies):
    retriever_dict = {}
    
    for company in companies:
        with open(f"dbs/{get_store_name(company)}.db", "rb") as f:
            db_bytes = pickle.load(f)
        
        db_temp = FAISS.deserialize_from_bytes(db_bytes, OpenAIEmbeddings(),
                                      allow_dangerous_deserialization=True)
        
        retriever_dict[company] = db_temp.as_retriever()
    
    return retriever_dict

In [165]:
retriever_dict = get_retriever_dict(df_sample.name.to_list())

In [166]:
def get_context(query, company):
    # TODO add sophisticated retriever for tables?
    return retriever_dict[company].get_relevant_documents(query)

## Structured inference

In [176]:
# MODEL = "gpt-4o-mini-2024-07-18"
MODEL = "gpt-4o-2024-08-06"

# load system prompt from .md
with open("instructions/answer_guidelines.md") as f:
    ANSWER_GUIDELINES = f.read()
    
with open("instructions/fin_info.md") as f:
    FIN_INFO = f.read()


system_prompt = f"""
You are an intelligent assistant tasked with retrieving and analyzing company data to answer specific questions based on the provided CONTEXT.

Your Objectives:
    - Chain of Thought: Document your reasoning process in 'chain_of_thought', including all intermediary steps. If your analysis involves calculations (e.g., ratios), explicitly state the numerator and denominator.
    - Handling Temporal Data: If the temporal scope of the data isn't an exact match (e.g., full-year data instead of Q4), use the most relevant available data to formulate your response. If specific quarterly data is requested but only full-year data is available, still provide an answer based on the available information, ensuring the CONTEXT contains sufficient relevant data.
    - Final Answer: Based on your reasoning, provide a final answer in 'answer' that strictly adheres to the guidelines outlined in. Accuracy and format compliance are crucial: \n {ANSWER_GUIDELINES} \n

Format:
    - chain_of_thought: Your reasoning process, including intermediary steps
    - answer: Your final answer following the answer guidelines

CONTEXT: 
<CONTEXT>
"""
   
# f"Your primary task is to accurately identify and extract specific financial metrics, ratios, and counts, even when synonymous "
# f"or contextually similar terms are used. Here is some useful information about these: \n {FIN_INFO}"

In [168]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class AnswerEvent(BaseModel):
    chain_of_thought: str
    answer: int | float | str


def ask_company_expert(query, company):
    print("### ASKING COMPANY EXPERT:", company)
    context = get_context(query, company)
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt.replace("<CONTEXT>", str(context))},
            {"role": "user", "content": query},
        ],
        response_format=AnswerEvent,
    )
    
    return completion.choices[0].message.parsed

In [104]:
ask_company_expert("How many assets does 'Accuray Incorporated' have in 2022 in dollars?", "Accuray Incorporated")  # 472849000

### ASKING COMPANY EXPERT: Accuray Incorporated


AnswerEvent(chain_of_thought="The document provides summarized balance sheet data for a joint venture (JV) as of March 31, 2022, and March 31, 2021. However, this data is specific to the JV and not Accuray Incorporated itself. The document lists 'Total liabilities and stockholders’ equity' as $472,849 for Accuray as of June 30, 2022. Typically, this total includes both total liabilities and total stockholders' equity, which together reflect the total assets of a company. Therefore, the total assets for Accuray Incorporated in 2022 should be the same as the total liabilities and stockholders’ equity, which is $472,849,000.", answer=472849000)

In [96]:
ask_company_expert("How many assets does 'Ethernity Networks Ltd' have in 2022 in dollars?", "Ethernity Networks Ltd")

['Ethernity Networks Ltd']


AnswerEvent(chain_of_thought='The provided context does not contain specific information regarding the total assets of Ethernity Networks Ltd for the year 2022. Therefore, I cannot provide an answer as the necessary data is absent.', answer='n/a')

In [105]:
ask_company_expert("What are the total R&D expenses of 'Ethernity Networks Ltd' in 2021?", "Ethernity Networks Ltd")

### ASKING COMPANY EXPERT: Ethernity Networks Ltd


AnswerEvent(chain_of_thought='The provided context does not contain specific figures or data detailing the research and development (R&D) expenses for Ethernity Networks Ltd for the year 2021. Without this information, I cannot accurately determine the R&D expenses for 2021.', answer='n/a')

## Multiple agents

In [177]:
# DELEGATION MANAGER

INVOLVED_COMPANIES = df_sample.name.to_list()

system_prompt_delegation = f"""
You will receive queries related to one or more companies. You have access to specialized agents capable of retrieving data for each specific company. To delegate tasks to these agents, follow these guidelines:

    1. Identify Companies: Extract the exact company names from the query and list them under 'companies'. Ensure that the company names match exactly with those provided in {INVOLVED_COMPANIES}. If the query mentions companies that are not in this list, enter 'SKIP' for both 'companies' and 'queries', and note this in the 'chain_of_thought'. If the company name in the query is incomplete (e.g., missing 'Inc.', 'Ltd.'), attempt to match it with the correct company name from the list.

    2. Formulate Queries: 
        - For each company identified, create a precise query that the specialized agents can use to retrieve the required information. 
        - List these queries under 'queries'. 
        - Add some information on where in annual company reports the specialized agents can usually find the information. 
        - If the query involves ratios, comparisons, or requires multiple metrics, suggest how the specialized agents should approach the problem to ensure accurate results. For instance, if calculating a ratio, indicate which metrics to retrieve and compare.
        - If using abbreviations also mention the full phrase or name ((e.g., for 'R&D' also use 'research and development').
        - If the query asks specifically for a quarterly data, also ask to provide the best available data if only full-year data is available. Be transparent about it in the 'chain_of_thought'.
    
    4. Chain of Thought: Provide a 'chain_of_thought' explaining how you identified the companies, formulated the queries, and any assumptions or steps taken to reach your final output. If you marked any queries as 'SKIP', explain why.
"""


class DelegationRequest(BaseModel):
    chain_of_thought: str
    companies: list[str]
    queries: list[str]


def ask_delegation_manager(query):    
    print("### ASKING DELEGATION MANAGER: ")
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_delegation},
            {"role": "user", "content": query},
        ],
        response_format=DelegationRequest,
    )
    
    return completion.choices[0].message.parsed

In [62]:
ask_delegation_manager("Has 'Ethernity Networks Ltd' more liabilities than 'Accuray Incorporated' in 2022?")

### ASKING DELEGATION MANAGER: 


DelegationRequest(chain_of_thought="In this query, I identified two companies: 'Ethernity Networks Ltd' and 'Accuray Incorporated'. Both of these companies are part of the provided list, allowing me to formulate specific queries for retrieving the necessary financial data. The query asks for a comparison of liabilities for the year 2022, which requires extracting total liabilities from the annual reports for both companies. I will retrieve the total liabilities for both companies from their 2022 reports to facilitate this comparison.", companies=['Ethernity Networks Ltd', 'Accuray Incorporated'], queries=['Retrieve total liabilities for Ethernity Networks Ltd from the 2022 annual report, typically found in the balance sheet section.', 'Retrieve total liabilities for Accuray Incorporated from the 2022 annual report, typically found in the balance sheet section.'])

In [34]:
ask_delegation_manager("How many assets does 'Ethernity Networks Ltd' have in 2022 in dollars?")

### ASKING DELEGATION MANAGER: 


DelegationRequest(chain_of_thought="The query explicitly mentions 'Ethernity Networks Ltd', which is included in the list of companies I have access to. I formulated the query to retrieve the specific information about the assets of 'Ethernity Networks Ltd' for the year 2022, specifying the currency as dollars so the specialized agents can provide a precise answer. There are no other companies mentioned in the query, so this is straightforward.", companies=['Ethernity Networks Ltd'], queries=['How many assets does Ethernity Networks Ltd have in 2022 in dollars? This information can usually be found in the balance sheet.'])

In [90]:
ask_delegation_manager("Did Global Medical REIT Inc. have a greater Debt-to-Equity ratio than Zegona Communications plc in Q2 2021?")

### ASKING DELEGATION MANAGER: 


DelegationRequest(chain_of_thought="The query mentions two companies: Global Medical REIT Inc. and Zegona Communications plc. Both are present in the provided list, so they are captured correctly. The query is asking for a comparison of the Debt-to-Equity ratio between these two companies for Q2 2021. To address this, the specialized agents should retrieve the Debt-to-Equity ratio for each company from their respective Q2 2021 financial reports. This is typically found in the financial statements section, specifically the balance sheet and notes on financial metrics. The agents should ensure that the data is from Q2 2021 to make a precise comparison. Hence, queries are formulated to retrieve this specific data from each company's financial reports for that period.", companies=['Global Medical REIT Inc.', 'Zegona Communications plc'], queries=['Retrieve the Debt-to-Equity ratio for Global Medical REIT Inc. from Q2 2021 financial reports.', 'Retrieve the Debt-to-Equity ratio for Zegona C

In [178]:
# EXECUTION MANAGER

system_prompt_execution = f"""
You are an intelligent assistant tasked with synthesizing a final answer to the USER_QUERY using specific company data provided in the CONTEXT. The CONTEXT includes information generated by expert systems along with their reasoning process.

Your Objectives:
    - Final Answer Construction: Your primary task is to deliver a final, accurate response to the USER_QUERY based on the provided CONTEXT.
    - Handling Temporal Data: If the temporal scope of the data isn't an exact match (e.g., full-year data instead of Q4), use the best available data to formulate your answer. If specific quarterly data is requested but only full-year data is available, still provide an answer based on the available information, ensuring the CONTEXT contains sufficient relevant data. 
    - Chain of Thought: Document your reasoning process in 'chain_of_thought', including intermediary steps and considerations that led to your final answer.
    - Strict Adherence: The final answer must be compliant with the specific guidelines. Accuracy, clarity, and adherence to these guidelines are critical: \n {ANSWER_GUIDELINES} \n 

Format:
    - chain_of_thought: Your reasoning process, including intermediary steps
    - answer: Your final answer following the guidelines. It should be of format: <OUTPUT_TYPE>, e.g., 'name', 'number' (including 0), 'boolean', etc.

Provided Inputs:

USER_QUERY: \n <QUERY> \n
CONTEXT: \n <CONTEXT>
"""

class ExecutionRequest(BaseModel):
    chain_of_thought: str
    answer: int | float | str

def ask_execution_manager(query, context, output_type):
    system_prompt_execution_full = system_prompt_execution.replace("<QUERY>", query).replace("<CONTEXT>", context).replace("<OUTPUT_TYPE>", output_type)
    
    print("### ASKING EXECUTION MANAGER: ")

    print("Context:", context)
    
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_execution_full},
            {"role": "user", "content": query},
        ],
        response_format=ExecutionRequest,
    )
    
    return completion.choices[0].message.parsed

In [179]:
# FINAL PIPELINE

def ask_question(query, output_type):
    print("## Query:", query)
    delegations = ask_delegation_manager(query)
    print("Delegations: \n", delegations)
    
    expert_context = {}
    for company, company_query in zip(delegations.companies, delegations.queries):
        if company.lower() == "skip" or company_query.lower() == "skip":
            expert_context[company] = "No data available"
        else:
            expert_context[company] = ask_company_expert(company_query, company)
    
    final_answer = ask_execution_manager(query, str(expert_context), output_type)
    
    return final_answer

In [112]:
ask_question("Who is the CEO of Tesla Inc.?", "name") # n/a

## Query: Who is the CEO of Tesla Inc.?
### ASKING DELEGATION MANAGER: 
Delegations: 
 chain_of_thought="Tesla Inc. is not in the list of companies that we are equipped to handle. Hence, according to the guidelines, the appropriate action is to mark both 'companies' and 'queries' as 'SKIP'." companies=['SKIP'] queries=['SKIP']
### ASKING EXECUTION MANAGER: 
Context: {'SKIP': 'No data available'}


ExecutionRequest(chain_of_thought='The context indicates there\'s no specific data available from the expert system about the CEO of Tesla Inc. As per the guidelines, if the needed information isn\'t available in the provided context, I must respond with "n/a".', answer='n/a')

In [113]:
ask_question("Who is the CEO of Ethernity Networks?", "name") # David Levi

## Query: Who is the CEO of Ethernity Networks?
### ASKING DELEGATION MANAGER: 
Delegations: 
 chain_of_thought="The query mentions the company 'Ethernity Networks' which matches exactly with the company 'Ethernity Networks Ltd' from the provided list. Therefore, the relevant company is identified, and I can proceed to form a query for this specific company to retrieve information about the CEO." companies=['Ethernity Networks Ltd'] queries=["Retrieve the name of the current CEO of Ethernity Networks Ltd. This information is typically found in the Management section of the company's annual report or on the company's official website under leadership or management team information."]
### ASKING COMPANY EXPERT: Ethernity Networks Ltd
### ASKING EXECUTION MANAGER: 
Context: {'Ethernity Networks Ltd': AnswerEvent(chain_of_thought='To find the name of the current CEO of Ethernity Networks Ltd., I will search the provided CONTEXT for any mention of executive roles, particularly looking for "

ExecutionRequest(chain_of_thought='The CONTEXT provides the name of the current CEO of Ethernity Networks. By examining the company\'s executive roles, specifically searching for titles like "CEO" or "Chief Executive Officer," it identifies "David Levi" as the CEO.', answer='David Levi')

In [114]:
ask_question("Has 'Ethernity Networks Ltd' more liabilities than 'Accuray Incorporated' in 2022?", "boolean") # Accuray: 419,660,000$, Ethernity: 12,257,291$ -> no is correct answer

## Query: Has 'Ethernity Networks Ltd' more liabilities than 'Accuray Incorporated' in 2022?
### ASKING DELEGATION MANAGER: 
Delegations: 
 chain_of_thought="First, I identified the companies mentioned in the query, which are 'Ethernity Networks Ltd' and 'Accuray Incorporated'. Both of these companies exist in the provided list. The query requests a comparison of liabilities between the two companies for the year 2022.\n\nTo address this query, I need to formulate precise questions for each company's specialized agent, asking them to retrieve the total liabilities from their respective 2022 financial statements, such as the balance sheet.\n\nFinally, I ensure that the agents understand to compare the total liabilities of each company for 2022, so that the initial query can be answered accurately." companies=['Ethernity Networks Ltd', 'Accuray Incorporated'] queries=['Retrieve the total liabilities for Ethernity Networks Ltd from the 2022 balance sheet.', 'Retrieve the total liabilities

ExecutionRequest(chain_of_thought='The context provides liability data for Accuray Incorporated for June 30, 2022, computed to be $419,660. However, for Ethernity Networks Ltd, there is no complete liability figure given for 2022. The context mentions that the financial information for Ethernity Networks is not aggregated into a total liabilities figure for the year 2022. \n\nThus, without a total liabilities figure for Ethernity Networks Ltd, I cannot compare the liabilities of Ethernity Networks Ltd to Accuray Incorporated.', answer='n/a')

In [149]:
ask_question('Did "Global Medical REIT Inc." have a greater Debt-to-Equity ratio than "Zegona Communications plc" in Q2 2021?', "boolean")

## Query: Did "Global Medical REIT Inc." have a greater Debt-to-Equity ratio than "Zegona Communications plc" in Q2 2021?
### ASKING DELEGATION MANAGER: 
Delegations: 
 chain_of_thought="The companies mentioned in the query are 'Global Medical REIT Inc.' and 'Zegona Communications plc', both of which are present in the provided list. Thus, the task should not be marked as 'SKIP'. The query involves calculating the Debt-to-Equity ratio for both companies in Q2 2021 to compare which one is greater. Typically, the Debt-to-Equity ratio can be found in the financial statements section of a company's quarterly report. Specifically, it requires extracting the total liabilities and shareholder's equity from the balance sheet. If only full-year data is available for 2021, use it as the best available data. The agent should ensure to retrieve the information accurately for the specified quarter and perform the division to find the Debt-to-Equity ratio for both companies before comparing them." c

ExecutionRequest(chain_of_thought="We are tasked with comparing the Debt-to-Equity ratios of 'Global Medical REIT Inc.' and 'Zegona Communications plc'. The CONTEXT does not provide specific Q2 2021 data, so we use the year-end 2021 data as an approximation.\n\n1. Global Medical REIT Inc.:\n   - Total liabilities at year-end 2021: $625,908,000\n   - Total equity at year-end 2021: $637,577,000\n   - Debt-to-Equity Ratio = 625,908,000 / 637,577,000 ≈ 0.9818\n\n2. Zegona Communications plc:\n   - Total liabilities at year-end 2021: 726 (in thousands)\n   - Total equity at year-end 2021: 14,654 (in thousands)\n   - Debt-to-Equity Ratio = 726 / 14,654 ≈ 0.0496\n\nWe compare the two calculated ratios: 0.9818 (Global Medical REIT Inc.) and 0.0496 (Zegona Communications plc). Clearly, 0.9818 is greater than 0.0496, therefore 'Global Medical REIT Inc.' has a greater Debt-to-Equity ratio than 'Zegona Communications plc' based on the data closest to Q2 2021 available (year-end 2021).", answer='ye

In [180]:
ask_question("What are the total R&D expenses of 'Ethernity Networks Ltd' in 2021?", "number")

## Query: What are the total R&D expenses of 'Ethernity Networks Ltd' in 2021?
### ASKING DELEGATION MANAGER: 
Delegations: 
 chain_of_thought="The query specifically mentions 'Ethernity Networks Ltd', which matches exactly with one of the company names in the provided list. The task is to find R&D expenses for 2021. Generally, such data is found in the financial section of the company's annual report, usually under a section dedicated to operating expenses or a summary of financials. I'll formulate a query for 'Ethernity Networks Ltd' to retrieve the required data." companies=['Ethernity Networks Ltd'] queries=['Retrieve the total research and development (R&D) expenses for Ethernity Networks Ltd for the fiscal year 2021. This information can often be found in the operating expenses section or the notes to the financial statements in the annual report.']
### ASKING COMPANY EXPERT: Ethernity Networks Ltd
### ASKING EXECUTION MANAGER: 
Context: {'Ethernity Networks Ltd': AnswerEvent(cha

ExecutionRequest(chain_of_thought='The CONTEXT provides information from the annual reports and financial statements for Ethernity Networks Ltd for the fiscal years 2021 and 2022. However, it explicitly mentions that there is no detailed breakdown of the R&D expenses for 2021 mentioned in the documents provided. Without the specific data, I cannot determine the R&D expenses for that year.', answer='n/a')

## Obtain final results 

In [173]:
# 5 min runtime for 40 questions

results = []

with open('data/questions.json', 'r') as json_file:
    items = json.load(json_file)
    for item in items:
        question = item["question"]
        response = ask_question(question, item["schema"]) # TODO could introduce schema as Daniel with arg=item["schema"] and classes/pydantic
        print(response)
        results.append({
            "question": question,
            "schema": item["schema"],
            "answer": response.answer
        })

with open('results/results_v7.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)

## Query: How much did "Accuray Incorporated" spend on risk management in Q2 2022?
### ASKING DELEGATION MANAGER: 
Delegations: 
 chain_of_thought="I identified the company 'Accuray Incorporated' from the query, which matches exactly with one of the companies from the specified list. The query specifically asks for spending on risk management in Q2 2022. To find this information, I formed a query asking the agent to look into the company's Q2 2022 financial reports, likely found in the management discussion and notes section of their quarterly filings, which detail various expenses including risk management costs. Additionally, I suggested checking the full-year data if quarterly specifics aren't available." companies=['Accuray Incorporated'] queries=['Retrieve the Q2 2022 financial report for Accuray Incorporated and identify the expenditure on risk management. Check sections related to management discussions and expense notes for detailed spending. If Q2 data is not available, provid

In [126]:
results

[{'question': 'How much did "Accuray Incorporated" spend on risk management in Q2 2022?',
  'schema': 'number',
  'answer': 'n/a'},
 {'question': 'Who is the CEO in the company "Zegona Communications plc"?',
  'schema': 'name',
  'answer': "Eamonn O'Hare"},
 {'question': 'Did "Global Medical REIT Inc." have a greater Debt-to-Equity ratio than "Zegona Communications plc" in Q2 2021?',
  'schema': 'boolean',
  'answer': 'yes'},
 {'question': 'How many stores did "Accuray Incorporated" have in the end of fiscal year 2021?',
  'schema': 'number',
  'answer': 'n/a'},
 {'question': 'How much did "Sandwell Aquatics Centre" spend on R&D in Q2 2023?',
  'schema': 'number',
  'answer': 'n/a'},
 {'question': 'Who is the CFO in the company "EMT"?',
  'schema': 'name',
  'answer': 'n/a'},
 {'question': 'Did "Calyxt, Inc." have a greater Return on Assets (ROA) than "Global Medical REIT Inc." in Q2 2023?',
  'schema': 'boolean',
  'answer': 'n/a'},
 {'question': 'How many stores did "OTC_UNCIF" have 

## OLD Inference with chain OLD

In [7]:
# Create vector store
# https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/vectorstore/
# Load documents from the specified folder
documents = load_documents_from_folder(SAMPLES_PATH, df_sample.sha1.to_list())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100) # TODO more sophisticated
texts = text_splitter.split_documents(documents)

if len(texts) == 0:
    raise ValueError("No text found in the specified folder")

In [8]:
embeddings = OpenAIEmbeddings() # deprecated?
db = FAISS.from_documents(texts, embeddings)
retriever = db.as_retriever()

In [54]:
# Store vector database
# with open("data/first_sample_db_3", "wb") as f:
#     pickle.dump(db.serialize_to_bytes(), f)

In [8]:
# Load vector database
# with open("data/vector_db_uni", "rb") as f:
#     db_bytes = pickle.load(f)
#     db = FAISS.deserialize_from_bytes(db_bytes, OpenAIEmbeddings(),
#                                       allow_dangerous_deserialization=True)
#     retriever = db.as_retriever()

In [31]:
llm = ChatOpenAI(temperature=0)
# llm = llm.bind(logprobs=True, top_logprobs=3)
# could try local models as well

system_prompt_2 = (
    f"You are an intelligent assistant tasked with answering questions based on specific company data provided in the context. "
    f"Please provide a chain of thought on how you arrived at your final answer in 'chain_of_thought'. \n"
    f"Then give a final answer in 'answer' where you strictly adhere to the following guidelines: \n {ANSWER_GUIDELINES} \n "
    f"Ensure that your answers are strictly compliant with these guidelines. Accuracy and adherence to the format are crucial. \n"
    
    # f"Your primary task is to accurately identify and extract specific financial metrics, ratios, and counts, even when synonymous "
    # f"or contextually similar terms are used. Here is some useful information about these: \n {fin_info}"
    
    "Context: \n {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_2),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)


# Function to ask a question
def ask_question(query):
    # retriever = get_retriever(query)
    
    chain = create_retrieval_chain(retriever, question_answer_chain)
    
    response = chain.invoke({"input": query})
    answer = response["answer"]
    print(f"Question: {query}\nAnswer: {answer}")

```python
    ('What was the {fin_metric} of "{company}" in {time_frame}?', "number"),
    ('How much did "{company}" spend on {focus_area} in {time_frame}?', "number"),
    ('What was the {ratio_or_metric} of "{company}" in {time_frame}?', "number"),
    ('How many {count_metric} did "{company}" have in {time_frame}?', "number"),
    ('Which company had a higher {fin_metric}: "{company1}", "{company2}" or "{company3}", in {time_frame}?', "name"),
    ('Did "{company1}" have a greater {ratio_or_metric} than "{company2}" in {time_frame}?', "boolean"),
    ('How much more did "{company1}" spend on {focus_area} compared to "{company2}" in {time_frame}?', "number"),
    ('Who is the {role} in the company "{company}"?', "name"),
```

In [ ]:
# multiple agents
# retriever per company
# -> first check if we need several processes to answer the questions (e.g. check revenues of 3 companies, then use these values with a final agent)
# chain of though: output json with chain of thought and final answer

In [28]:
# Ask a question
ask_question("How many assets does 'Accuray Incorporated' have in 2022 in dollars?") # 472.849 

Question: How many assets does 'Accuray Incorporated' have in 2022 in dollars?
Answer: chain_of_thought: 
1. Look for the Consolidated Balance Sheets section to find the total assets for Accuray Incorporated in 2022.
2. Identify the value listed under "Total current assets" and any other asset categories to calculate the total assets for the company in 2022.

answer: 352890


In [29]:
ask_question("How many assets does 'Accuray Incorporated' have in 2021 in dollars?") # 480.098

Question: How many assets does 'Accuray Incorporated' have in 2021 in dollars?
Answer: Chain of Thought:
1. Look for the Consolidated Balance Sheets section.
2. Find the total assets value for 'Accuray Incorporated' for June 30, 2021.

Answer:
352773


In [32]:
ask_question("How many liabilities does 'Accuray Incorporated' have in 2021 in dollars?") # 411.258

Question: How many liabilities does 'Accuray Incorporated' have in 2021 in dollars?
Answer: Chain of Thought:
1. Liabilities can be calculated by subtracting total assets from total equity, as liabilities = assets - equity.
2. The total assets for 2021 are not provided directly but can be calculated by summing up the current assets and other assets like property, equipment, etc.
3. The total equity for 2021 is not directly given but can be calculated by adding common stock, additional paid-in capital, accumulated other comprehensive income (loss), and accumulated deficit from the stockholders' equity statement.
4. Once total assets and total equity are calculated, liabilities can be determined.

Answer:
$378,000

Chain of Thought:
- Total assets for 2021 are calculated as $350,890 (current assets) + $12,685 (property and equipment) + $13,879 (investment in joint venture) + $16,798 (operating lease right-of-use assets) + $57,840 (goodwill) + $250 (intangible assets) = $452,342.
- Total 

In [78]:
ask_question("How many stores did 'Strike Energy Limited' have in the end of fiscal year 2021?") # n/a

Question: How many stores did 'Strike Energy Limited' have in the end of fiscal year 2021?
Answer: n/a


In [83]:
# Total R&D Expenses of Ethernity Networks
ask_question("What are the total R&D expenses of 'Ethernity Networks' in 2021?") # 5 550 912

ask_question("What are the total R&D expenses of 'Ethernity Networks' in 2022?") # 6 618 795

Question: What are the total R&D expenses of 'Ethernity Networks' in 2021?
Answer: 5550912
Question: What are the total R&D expenses of 'Ethernity Networks' in 2022?
Answer: 6618795
